In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = model.with_structured_output(Joke)

In [9]:
structured_llm.invoke("Tell me a joke about cats")

Joke(setup='Why was the cat sitting on the computer?', punchline='To keep an eye on the mouse!')

In [11]:
from typing import List
from typing import Union

    
class Article(BaseModel):
    link: Union[str|None]
    title: Union[str|None]
    source: Union[str|None]
    date: Union[str|None]
    snippet: Union[str|None]
    summary: Union[str|None]
    
class Topic(BaseModel):
    topic: str = Field(description="The topic name")
    articles: List[Article] = Field(description="The list of related news articles")
    
    
class NewsClusterer(BaseModel):
    topics: List[Topic] = Field(description="The list of generated topics")

In [12]:
structured_llm = model.with_structured_output(NewsClusterer)

In [14]:
structured_llm.invoke([
    (
        "system",
        "'You are a news clustering generator. You will cluster news articles into related topics based on their title and summary.'",
    ),
    ("human", "Articles:\n\n Article Title: Two inches of rain per hour possible: Flood watch issued by National Weather Service\nArticle Snippet: Two inches of rain per hour possible: Flood watch issued by National Weather ServiceNational Weather Service has issued a time sensitive severe alert. A flood watch is in effect Tuesday from 5 p.m. to 11 p.m. this evening. WHERE: Portions of central, northern, northwest, and western Virginia, including the following...\n-----\nArticle Title: National Weather Service issues flood watch for Tri-Cities until 10 p.m. Saturday\nArticle Snippet: National Weather Service issues flood watch for Tri-Cities until 10 p.m. SaturdaySaturday's hard rain pummeling the Tri-City area has led to a flash-flood watch from the National Weather Service until 10 p.m., with warnings issues for points north and east of here. NWS' Wakefield office issued the advisory around 3:40...\n-----\nArticle Title: 'Life-Threatening Flash Flooding' Possible In GA; Warnings Issued\nArticle Snippet: Weather 'Life-Threatening Flash Flooding' Possible In GA; Warnings Issued \"Life-threatening flash flooding\" may occur in some parts of Georgia on Tuesday, according to two warnings. A flash flood warning has been issued for some Georgia cities until 10 p.m. Tuesday. (Shutterstock)GEORGIA — A flash flood warning has been issued for...\n-----\nArticle Title: Flash flood warning issued for Harrisonburg and Rockingham County\nArticle Snippet: HARRISONBURG, Va. ( ROCKTOWN NOW ) – The National Weather Service has issued a flash flood warning for central Rockingham County and Harrisonburg, Virginia until 7:45 p.m. According to an alert from NWS in Sterling, Va., thunderstorms are producing heavy rainfall of 1-2 inches per hour, with 1-1.5 inches already..."),
])

NewsClusterer(topics=[Topic(topic='Flood Warnings by National Weather Service', articles=[Article(link=None, title='Two inches of rain per hour possible: Flood watch issued by National Weather Service', source='Unknown', date='Unknown', snippet='Two inches of rain per hour possible: Flood watch issued by National Weather ServiceNational Weather Service has issued a time sensitive severe alert. A flood watch is in effect Tuesday from 5 p.m. to 11 p.m. this evening. WHERE: Portions of central, northern, northwest, and western Virginia, including the following...', summary='National Weather Service has issued a flood watch for central, northern, northwest, and western Virginia with the possibility of two inches of rain per hour.'), Article(link=None, title='National Weather Service issues flood watch for Tri-Cities until 10 p.m. Saturday', source='Unknown', date='Unknown', snippet="National Weather Service issues flood watch for Tri-Cities until 10 p.m. SaturdaySaturday's hard rain pummel

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI


class PrimaryKeywordGenerator(BaseModel):
    # """A component responsible for generating a primary keyword for an article starting from a list of related articles."""
    # primary_keyword: PrimaryKeyword = Field(
    #     description="The generated primary keyword"
    # )
    primary_keyword: str = Field(
        description="The generated primary keyword"
    )

primary_keyword_gen_system = """You are a primary keyword generator. You will generate a primary keyword to guide the writing of an SEO-efficient article starting from a list of articles."""
primary_keyword_gen_human = """Articles:\n\n {articles}"""
articles = 'Title: Two inches of rain per hour possible: Flood watch issued by National Weather Service\nSnippet: Two inches of rain per hour possible: Flood watch issued by National Weather ServiceNational Weather Service has issued a time sensitive severe alert. A flood watch is in effect Tuesday from 5 p.m. to 11 p.m. this evening. WHERE: Portions of central, northern, northwest, and western Virginia, including the following...\n----\nTitle: National Weather Service issues flood watch for Tri-Cities until 10 p.m. Saturday\nSnippet: National Weather Service issues flood watch for Tri-Cities until 10 p.m. SaturdaySaturday\'s hard rain pummeling the Tri-City area has led to a flash-flood watch from the National Weather Service until 10 p.m., with warnings issues for points north and east of here. NWS\' Wakefield office issued the advisory around 3:40...\n----\nTitle: \'Life-Threatening Flash Flooding\' Possible In GA; Warnings Issued\nSnippet: Weather \'Life-Threatening Flash Flooding\' Possible In GA; Warnings Issued "Life-threatening flash flooding" may occur in some parts of Georgia on Tuesday, according to two warnings. A flash flood warning has been issued for some Georgia cities until 10 p.m. Tuesday. (Shutterstock)GEORGIA — A flash flood warning has been issued for...\n----\nTitle: Flash flood warning issued for Harrisonburg and Rockingham County\nSnippet: HARRISONBURG, Va. ( ROCKTOWN NOW ) – The National Weather Service has issued a flash flood warning for central Rockingham County and Harrisonburg, Virginia until 7:45 p.m. According to an alert from NWS in Sterling, Va., thunderstorms are producing heavy rainfall of 1-2 inches per hour, with 1-1.5 inches already...'
primary_keyword_gen_human = primary_keyword_gen_human.format(articles=articles)
# primary_keyword_gen_template = ChatPromptTemplate.from_messages(
#     [
#         ("system", primary_keyword_gen_system),
#         ("human", primary_keyword_gen_human)
#     ]
# )
primary_keyword_gen_template =[
        ("system", primary_keyword_gen_system),
        ("human", primary_keyword_gen_human)
    ]

structured_llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0).with_structured_output(PrimaryKeywordGenerator)
generator = primary_keyword_gen_template | structured_llm
# pk = generator.invoke({"articles": articles})
pk = structured_llm.invoke(primary_keyword_gen_template)
pk

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'list'>